In [17]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [3]:
dataset0 = pd.read_csv('data/dataset0.csv')
dataset0.fillna('0',inplace=True)
dataset0.label.replace(-1,0,inplace=True)
dataset1 = pd.read_csv('data/dataset1.csv')
dataset1.fillna('0',inplace=True)
dataset1.label.replace(-1,0,inplace=True)
dataset2 = pd.read_csv('data/dataset2.csv')
dataset2.fillna('0',inplace=True)
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('data/dataset3.csv')
dataset3.fillna('0',inplace=True)

dataset0.drop_duplicates(inplace=True)
dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset3.drop_duplicates(inplace=True)

In [8]:
dataset01 = pd.concat([dataset0,dataset1],axis=0)
dataset012 = pd.concat([dataset01,dataset2],axis=0)
dataset01.head()

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count_x,coupon_count_y,...,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,online_total_consume,label
0,1439408,0.950000,0,19,6,0,0,0,3,3,...,1,1,-1,-1,1,1,-1,-1,0,0
1,94107,0.900000,2,12,30,0,0,0,24448,24448,...,1,1,-1,-1,1,1,-1,-1,0,0
2,253750,0.750000,0,27,14,0,0,0,690,690,...,1,1,-1,-1,1,1,-1,-1,0,0
3,2881376,0.750000,0,21,8,0,0,0,690,690,...,1,1,-1,-1,1,1,-1,-1,0,1
4,4061024,0.833333,10,9,27,0,0,0,8209,8209,...,1,1,-1,-1,1,1,-1,-1,0,0


In [20]:
dataset01_y = dataset01.label
dataset01_x = dataset01.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)
dataset012_y = dataset012.label
dataset012_x = dataset012.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)

In [21]:
dataset2_y = dataset2.label
dataset2_x = dataset2.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)

In [22]:
dataset3_preds = dataset3[['user_id','coupon_id','date_received']]

In [23]:
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received','day_gap_before','day_gap_after'],axis=1)

In [ ]:
# for params test  网格搜索最优参数
param_test1 = {'min_samples_leaf':list(range(50,81,10))}
gsearch1 = GridSearchCV(estimator=GradientBoostingClassifier(
                                                               learning_rate=0.1,
                                                               n_estimators=100,
                                                               subsample=0.8,
                                                               max_features='sqrt',
                                                               max_depth=8,
                                                                verbose=1,
                                                               min_samples_split=130,
                                                               # min_samples_leaf=30,
                                                               # min_weight_fraction_leaf=，
                                                               random_state=10,
                                                               ),
                        param_grid=param_test1,
                        scoring='roc_auc',
                        iid=False,
                        cv=5
)
gsearch1.fit(dataset01_x,dataset01_y)
print(gsearch1.cv_results_)
print(gsearch1.best_params_, gsearch1.best_score_)
print(roc_auc_score(dataset2_y,gsearch1.predict_proba(dataset2_x)[:,1]))

####for train
gbm0 = GradientBoostingClassifier(max_features='sqrt',
                                  learning_rate=0.1,
                                  subsample=0.8,
                                  random_state=10,
                                  n_estimators=100,
                                  min_samples_split=300,
                                  max_depth=13,
                                  verbose=1,
                                  min_samples_leaf=30
)
gbm0.fit(dataset01_x,dataset01_y)
y_predprob = gbm0.predict_proba(dataset2_x)[:,1]
print(roc_auc_score(dataset2_y,y_predprob))

#for submit
gbm0 = GradientBoostingClassifier(max_features='sqrt',
                                  learning_rate=0.1,
                                  min_samples_leaf=20,
                                  subsample=0.8,
                                  verbose=1,
                                  random_state=10,
                                  n_estimators=100)
gbm0.fit(dataset012_x,dataset012_y)
dataset3_preds['label'] = gbm0.predict_proba(dataset3_x)[:,1]
dataset3_preds.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds.to_csv("gbdt_preds.csv",index=None,header=None)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.5413           0.0333           55.65s
         2           0.5198           0.0228           53.82s
         3           0.5010           0.0175           53.75s
         4           0.4908           0.0123           54.61s
         5           0.4756           0.0107           52.83s
         6           0.4637           0.0094           52.03s
         7           0.4590           0.0073           50.40s
         8           0.4510           0.0073           50.27s
         9           0.4434           0.0060           49.44s
        10           0.4379           0.0060           49.24s
        20           0.4057           0.0017           42.22s
        30           0.3909           0.0009           35.52s
        40           0.3824           0.0005           29.55s
        50           0.3769           0.0003           23.87s
        60           0.3737           0.0001           18.53s
       